In [7]:
#%cd source
%load_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import time
import spacy
import os
import pipeline
from spacy.language import Language
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Règles

In [8]:
# Ajouter une règle de segmentation personnalisée
def custom_segmentation(doc):
    to_end_after_bracket = False
    for i, token in enumerate(doc[:-1]):
        
        # si mot qui créer une phrase
        if token.text == "Id." or token.text == "See":
            token.is_sent_start = True
        
        if token.text == "Ibid." :
            token.is_sent_start = True
            if i < len(doc) - 2 :
                doc[i + 1].is_sent_start = True

        
        # si point
        if token.text == ".":
            token.is_sent_start = False               
            
            # Les parenthèses / crochets font partie de la phrase précédente
            if doc[token.i + 1].text == "[" or doc[token.i + 1].text == "(" or doc[token.i + 2].text == "[" or doc[token.i + 2].text == "(":
                doc[token.i + 1].is_sent_start = False
                to_end_after_bracket = True
            # Le point est suivi d'une minuscule   
            elif re.match(r'^[a-z]', doc[i + 1].text):
                doc[i + 1].is_sent_start = False
            
            # si ressemble au point d'un sigle
            elif (i > 0 and re.match(r'^[A-Z][A-Za-z]{0,3}$|^[a-z]{1,2}$', doc[i-1].text) and i < len(doc) - 2):
                doc[i + 1].is_sent_start = False
            
            # si dernier point des points de suspension    
            elif (i > 2 and doc[i - 1].text == "." and doc[i - 2].text == ".")  and i < len(doc) - 2:
                doc[i + 1].is_sent_start = False
                
            continue
        
        # si fermer parenthèse ou crochet et to_end_after_bracket est vrai
        if (token.text == "]" or token.text == ")") and to_end_after_bracket:
            # si suivi de parenthèses ou crochets, la phrase continue, sinon fin de phrase
            if (doc[i + 1].text != "(" or doc[i + 1].text != "["):
                doc[i + 1].is_sent_start = True
                to_end_after_bracket = False
            # si suivi ponctuation, la phrase continue
            elif (doc[i+1].text == "." or doc[i+1].text == ";" or token.text == ":"):
                to_end_after_bracket = False
            continue
        
        # si est entre parenthèse, continue la phrase    
        if to_end_after_bracket :
            doc[i + 1].is_sent_start = False
            continue
        
        # si virgule, la virgule fait partie de la phrase précédente et pas de fin de phrase
        if token.text == "," and i < len(doc) - 2 :
            token.is_sent_start = False
            doc[i + 1].is_sent_start = False
            continue
            
        # si ponctuation, la ponctuation fait partie de la phrase précédente
        if token.text == ";" or token.text == ":":
            token.is_sent_start = False
            # si suivie d'une minuscule, ne marque pas la fin de la phrase, sinon oui
            if (i < len(doc) - 2 and re.match(r'^[a-z]', doc[i + 1].text)):
                doc[i + 1].is_sent_start = False
            elif (i < len(doc) - 2 and doc[i + 1].text == "\""):
                doc[i + 1].is_sent_start = True
                if i < len(doc) - 3:
                    doc[i + 2].is_sent_start = False
            else :
                doc[i + 1].is_sent_start = True
            continue
            
    return doc

# Ajouter la fonction de segmentation personnalisée au pipeline spaCy
@Language.component("custom_segmentation")
def custom_segmentation_component(doc):
    return custom_segmentation(doc)

# Charger le modèle spaCy de base
nlp = spacy.load("en_core_web_sm")

# Insérer la segmentation personnalisée au début du pipeline
nlp.add_pipe("custom_segmentation", last=True)


<function __main__.custom_segmentation_component(doc)>

Règle qui ne marche pas : 
- Gestion des guillemets parce que s'il en manque un, tout est faussé parce que contrairement aux parenthèses/crochets, il y a une ambiguité sur le début ou la fin de citation
- Gestion des points-virgules (parfois fin de phrases, parfois non)

In [ ]:
# if token.text == "\"":
#     # ouverture de citation
#     if not in_quote :
#         in_quote = True
#         if i < len(doc) - 2 :
#             doc[i + 1].is_sent_start = False
        
#     # fermeture de citation 
#     else :
#         in_quote = False
#         token.is_sent_start = False
#         if i < len(doc) - 2 :
#             doc[i + 1].is_sent_start = True

### Évaluation sur train

In [14]:
res_eval_train = []
dir_path = "../documents/train/*"
models = ['spacy', 'custom_spacy']
tokenizers = ['spacy', 'spacy']
for i in range(len(models)):
    start = time.time()
    r, p, f = pipeline.evaluation(dir_path, tokenizers[i], models[i])
    res_eval_train += [[time.time()-start, r, p, f]]

0.6928104575163399 0.8712328767123287 0.7718446601941747 ../documents/train/opinion-G16-(Taylor v. Sturgell, 553 U.S. 880).txt
0.7477477477477478 0.83 0.7867298578199052 ../documents/train/ROSENBLATT v. BAER_MCL.txt
0.5 0.6141732283464567 0.5512367491166078 ../documents/train/opinion-G16-(Florida Power _ Light Co. v. Electric.txt
0.6265060240963856 0.8 0.7027027027027027 ../documents/train/opinion-G16-(Jenkins v. Georgia, 418 U.S. 153).htm.txt
0.6776315789473685 0.8373983739837398 0.749090909090909 ../documents/train/opinion-G16-(California v. Superior Court of Cal.,.txt
0.7063492063492064 0.717741935483871 0.712 ../documents/train/California v. Greenwood_retagged.txt
0.7386363636363636 0.8496732026143791 0.7902735562310031 ../documents/train/Commissioner v. Groetzinger_MCL_retagged.txt
0.6538461538461539 0.8686131386861314 0.7460815047021944 ../documents/train/opinion-G16-(United States v. Mississippi Chemical.txt
0.7079646017699115 0.898876404494382 0.7920792079207921 ../documents/tr

In [15]:
little_df = pd.DataFrame(res_eval_train,columns=["execution_time","precision", "recall", "F1_score"], index=['spacy', "custom_spacy"])
little_df.style.highlight_max(color = 'green', axis = 0).highlight_max(color = 'red', axis = 0, subset = ["execution_time"]).highlight_min(color = 'red', axis = 0).highlight_min(color = 'green', axis = 0, subset = ["execution_time"])

,execution_time,precision,recall,F1_score
spacy,1.475347,0.830381,0.705240,0.761185
custom_spacy,9.796435,0.940357,0.866743,0.901046


### Évaluation sur test

In [11]:
res_eval_test = []
dir_path = "../documents/test/*"
models = ['spacy', 'custom_spacy']
tokenizers = ['spacy', 'spacy']
for i in range(len(models)):
    start = time.time()
    r, p, f = pipeline.evaluation(dir_path, tokenizers[i], models[i])
    res_eval_test += [[time.time()-start, r, p, f]]

0.7105263157894737 0.8709677419354839 0.782608695652174 ../documents/test/opinion-G16-(Rogers v. Quan, 357 U.S. 193).html.txt
0.7990196078431373 0.7616822429906542 0.7799043062200957 ../documents/test/opinion-G16-(Return Mail, Inc. v. Postal Service,.txt


In [12]:
little_df = pd.DataFrame(res_eval_test,columns=["execution_time","precision", "recall", "F1_score"], index=['spacy', "custom_spacy"])
little_df.style.highlight_max(color = 'green', axis = 0).highlight_max(color = 'red', axis = 0, subset = ["execution_time"]).highlight_min(color = 'red', axis = 0).highlight_min(color = 'green', axis = 0, subset = ["execution_time"])

,execution_time,precision,recall,F1_score
spacy,0.342793,0.809799,0.770942,0.787504
custom_spacy,2.343848,0.874458,0.928753,0.899522


### Test finetune spacy (abandonné)

In [4]:
base_model = "en_core_web_sm"
nlp = spacy.load(base_model)

if "sentencizer" not in nlp.pipe_names:
    sentencizer = nlp.add_pipe("sentencizer")
    
# Préparation des données d'entraînement
def process_text_files(directory):
    data = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            with open(os.path.join(directory, filename), "r") as file:
                text = file.read()
                # Supprimer les sauts de ligne supplémentaires
                text = text.strip()
                # Séparer le texte en phrases
                sentences = text.split("\n")
                start_end_pos = []
                start_pos = 0
                for sentence in sentences:
                    end_pos = start_pos + len(sentence)
                    start_end_pos.append((start_pos, end_pos))
                    # Ajouter la longueur d'un saut de ligne
                    start_pos = end_pos + 1
                data.append((text, {"words": sentences}))
    return data

# Définir le répertoire contenant vos fichiers .txt
train_directory = "../documents/train/"

# Charger les données d'entraînement
train_data = process_text_files(train_directory)

nlp.disable_pipes("tagger", "parser")  # Désactiver le Tagger et le Parser par exemple

# Entraînement du modèle
# Utiliser vos données d'entraînement pour affiner le modèle
for text, annotations in train_data:
    # Créer un objet Example à partir du texte et des annotations
    example = spacy.training.Example.from_dict(nlp.make_doc(text), annotations)
    # Mettre à jour le modèle avec l'exemple
    nlp.update([example], losses={})

# Sauvegarder le modèle affiné
nlp.to_disk("../models/fine_tuned_spacy_model")

In [4]:
nlp = spacy.load("../models/fine_tuned_spacy_model")

In [5]:

doc = nlp("This is a sentence. [1] This is another sentence.")
sentences = []
for sentence in doc.sents:
    #print(sentence.text)
    sentences += [sentence.text]
    
    

This is a sentence. [
1] This is another sentence.


/home/labicquette/.local/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
